In [149]:
#Kimia Arjmand 4003623001
#Maryam Hoseinpoor 4003623012

In [ ]:
import pandas as pd
import networkx as nw
import sys
import time
from queue import PriorityQueue

In [150]:
df = pd.read_csv(r"C:\Users\Tank.bnd\Desktop\AI_project\search-blaster\Flight_Data1.csv")
df

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Pegasus Airlines,Sabiha Gökçen International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.898602,29.309200,312,Tehran,Iran,35.416100,51.152199,3305,1998.541333,2.624833,271.489760
1,Turkish Airlines,Atatürk International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.976898,28.814600,163,Tehran,Iran,35.416100,51.152199,3305,2040.978811,2.882362,300.589499
2,Emirates,Dubai International Airport,Imam Khomeini International Airport,Dubai,United Arab Emirates,25.252800,55.364399,62,Tehran,Iran,35.416100,51.152199,3305,1199.863567,1.536046,210.215879
3,Etihad Airways,Abu Dhabi International Airport,Imam Khomeini International Airport,Abu Dhabi,United Arab Emirates,24.433001,54.651100,88,Tehran,Iran,35.416100,51.152199,3305,1266.681453,2.144199,175.864733
4,Air Arabia,Sharjah International Airport,Imam Khomeini International Airport,Sharjah,United Arab Emirates,25.328600,55.517200,111,Tehran,Iran,35.416100,51.152199,3305,1196.902147,1.772627,174.332843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6831,Kenmore Air,Boeing Field King County International Airport,William R Fairchild International Airport,Seattle,United States,47.529999,-122.302002,21,Port Angeles,United States,48.120201,-123.500000,291,110.930397,0.161072,31.661891
6832,Kenmore Air,Boeing Field King County International Airport,Orcas Island Airport,Seattle,United States,47.529999,-122.302002,21,Eastsound,United States,48.708199,-122.910004,31,138.564899,0.899144,26.201671
6833,Kenmore Air,William R Fairchild International Airport,Boeing Field King County International Airport,Port Angeles,United States,48.120201,-123.500000,291,Seattle,United States,47.529999,-122.302002,21,110.930397,0.751768,99.285196
6834,Kenmore Air,Friday Harbor Airport,Boeing Field King County International Airport,Friday Harbor,United States,48.521999,-123.024002,113,Seattle,United States,47.529999,-122.302002,21,122.677708,1.119010,19.599675


In [151]:
df1 = df.drop_duplicates('SourceAirport')
df2 = df.drop_duplicates('DestinationAirport')
df1.set_index('SourceAirport',inplace=True)
df2.set_index('DestinationAirport',inplace=True)

In [152]:
G = nw.DiGraph()

In [153]:
G = nw.from_pandas_edgelist(df, source = 'SourceAirport', target = 'DestinationAirport', edge_attr = ['Distance','FlyTime','Price','SourceAirport_City','SourceAirport_Country','DestinationAirport_City','DestinationAirport_Country', 'SourceAirport_Latitude', 'SourceAirport_Longitude', 'SourceAirport_Altitude', 'DestinationAirport_Latitude','DestinationAirport_Longitude', 'DestinationAirport_Altitude', 'Airline'])

In [154]:
source_and_destination_airports = input()
airports = source_and_destination_airports.split(' - ')
start_node , target_node = airports[0],airports[1]

In [155]:
def dijkstra_algorithm(graph, start_node):
    unvisited_nodes = list(graph.nodes())
    shortest_path = {}
    previous_nodes = {}
    # using max_value to initialize the "infinity" value of the unvisited nodes   
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
    # initializing the starting node's value with 0   
    shortest_path[start_node] = 0
    while unvisited_nodes:
        current_min_node = None
        for node in unvisited_nodes:  # Iterate over the nodes
            if current_min_node is None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node
        # retrieving the current node's neighbors and updating their distances
        neighbors = list(graph.adj[current_min_node])
        for neighbor in neighbors:
            value = graph.get_edge_data(current_min_node, neighbor)
            tentative_value = shortest_path[current_min_node] + value.get('Distance')
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                # updating the best path to the current node
                previous_nodes[neighbor] = current_min_node
        unvisited_nodes.remove(current_min_node)
    return previous_nodes 

In [156]:
def heuristic(current, D_La, D_Lo, D_A):
    try:
        row = df1.loc[current]
        S_La = row['SourceAirport_Latitude']
        S_Lo = row['SourceAirport_Longitude']
        S_A = row['SourceAirport_Altitude']
        return ((S_La - D_La) ** 2 + (S_Lo - D_Lo) ** 2 + (S_A - D_A) ** 2) ** 0.5
    except:
        return sys.maxsize

In [157]:
def a_star_algorithm(graph, start, target):
    row_t = df2.loc[target]
    D_La = row_t['DestinationAirport_Latitude']
    D_Lo = row_t['DestinationAirport_Longitude']
    D_A = row_t['DestinationAirport_Altitude']

    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current = frontier.get()

        if current == target:
            return came_from
        neighbors = list(graph.adj[current])
        for neighbor in neighbors:
            value = graph.get_edge_data(current, neighbor)
            new_cost = cost_so_far[current] + value.get('Distance')
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                priority = new_cost + heuristic(neighbor, D_La, D_Lo, D_A)
                frontier.put(neighbor, priority)
                came_from[neighbor] = current
    return came_from

In [158]:
def calculate_result(previous_nodes, start_node, target_node):
    path = []
    node = target_node
    while node != start_node:
        path.append(node)
        node = previous_nodes[node]
 
    path.append(start_node)
    path.reverse()
    return path

In [159]:
def write_file(mode, algorithm, graph, path, execution_time):
    total_price = 0
    total_time = 0
    total_duration = 0
    f = open("Blaster-UIAI4021-PR1-Q1.txt", mode)
    f.write("{} Algorithm\n"
            "Execution Time: {}\n"
            ".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-\n".format(algorithm, execution_time))
    for i in range(len(path)-1):
        value = graph.get_edge_data(path[i], path[i + 1])
        total_price = total_price + value.get('Price')
        total_time = total_time + value.get('FlyTime')
        total_duration = total_duration + value.get('Distance')
        f.write("Flight #{0} ({10}):\n"
                "From: {2}-{1}, {3}\n"
                "To: {5}-{4}, {6}\n"
                "Duration: {7}km\n"
                "Time: {8}h\n"
                "Price: {9}$"
                .format(i + 1, value.get('SourceAirport_City'), path[i], value.get('SourceAirport_Country'),
                        value.get('DestinationAirport_City'), path[i + 1], value.get('DestinationAirport_Country'),
                        round(value.get('Distance'),2),
                        round(value.get('FlyTime'),2),
                        round(value.get('Price'),2),
                        value.get('Airline')))
        f.write("\n----------------------------\n")
    f.write("Total Price: {}$\n"
            "Total Duration: {}km\n"
            "Total Time: {}h\n\n".format(round(total_price,2), round(total_duration,2), round(total_time,2)))
    f.close()
    

In [160]:
dijkstra_start_time = time.time()
dijkstra_previous_nodes = dijkstra_algorithm(graph=G, start_node=start_node)
dijkstra_path = calculate_result(dijkstra_previous_nodes, start_node=start_node, target_node=target_node)
dijkstra_end_time = time.time()

In [161]:
a_star_start_time = time.time()
a_star_previous_nodes = a_star_algorithm(graph=G, start=start_node, target=target_node)
a_star_path = calculate_result(a_star_previous_nodes, start_node=start_node, target_node=target_node)
a_star_end_time = time.time()

In [162]:
write_file(mode='w',algorithm='Dijkstra',graph=G, path=dijkstra_path, execution_time = dijkstra_end_time - dijkstra_start_time)

write_file(mode='a',algorithm='A*',graph=G, path=a_star_path, execution_time = a_star_end_time - a_star_start_time)